In [1]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

In [2]:
import data_helpers

df = data_helpers.load_data(max_sentence_length=1000000,balance_classes=False,shuffle=True)

In [3]:
from tqdm import tqdm
df.text = df.text.astype(str)
words = []
for text in tqdm(df.text):
    words.extend(text.split(' '))

100%|██████████| 215702/215702 [00:00<00:00, 299373.66it/s]


In [4]:
print('Data size', len(words))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000


def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
          '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 10001

session = tf.Session(graph=graph)
# We must initialize all variables before we use them.
init.run(session=session)
print("Initialized")

average_loss = 0
for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
        if step > 0:
            average_loss /= 2000
        # The average loss is an estimate of the loss over the last 2000 batches.
        print("Average loss at step ", step, ": ", average_loss)
        average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
        sim = similarity.eval(session=session)
        for i in xrange(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = "Nearest to %s:" % valid_word
            for k in xrange(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = "%s %s," % (log_str, close_word)
            print(log_str)
final_embeddings = normalized_embeddings.eval(session=session)

# Step 6: Visualize the embeddings.

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

Data size 11249546
Most common words (+UNK) [['UNK', 46941], ('the', 567657), ('i', 378280), ('and', 323175), ('to', 287912)]
Sample data [310, 575, 14, 15, 377, 4, 28, 100, 1764, 166] ['pretty', 'gripping', 'read', 'that', 'seems', 'to', 'be', 'character', 'study', 'before']
575 gripping -> 310 pretty
575 gripping -> 14 read
14 read -> 575 gripping
14 read -> 15 that
15 that -> 14 read
15 that -> 377 seems
377 seems -> 4 to
377 seems -> 15 that
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step  0 :  277.306091309
Nearest to book: imprinted, dickenslike, sexthat, saccharine, mysteriesthrillersuspense, nerdie, selfcentred, mysterynoir,
Nearest to did: taste, barefoot, illmannered, yancey’s, grail, speared, arranges, hourly,
Nearest to couldnt: depths, trlogy, characterthe, searching, sequelill, perceived, nobodys, emerges,
Nearest to really: everdeen’s, degrades, fetishy, hangingwhich, proceeds, inti, emmulate, disbelieve,
Nearest

In [8]:
embeddings.eval(session=session).shape

(50000, 128)

In [10]:
dictionary

{'': 6,
 'fawn': 17590,
 'thesetoo': 40566,
 'bdsmso': 40559,
 'nunnery': 25882,
 'nondetective': 40561,
 'sowell': 30470,
 'askew': 35582,
 'woods': 3639,
 'clotted': 40563,
 'spiders': 22558,
 'hanging': 1243,
 'ultimatley': 25584,
 'woody': 12233,
 'trawling': 40564,
 'goodbetter': 37168,
 'comically': 12666,
 'suzane': 14239,
 'caner': 40567,
 'canes': 20469,
 'yearnine': 40568,
 'stringsplease': 40569,
 'acurately': 20470,
 'scold': 19627,
 'storiesit': 40570,
 'xandra': 7690,
 'originality': 4461,
 'suzans': 40560,
 'refunding': 40636,
 'caned': 40572,
 'fitest': 30475,
 'mivie': 40574,
 'hermann': 16101,
 'riiigghhttt': 40575,
 'storiesif': 40576,
 'waldenbooks': 35583,
 'wracked': 18186,
 'discribed': 14876,
 'thensex': 40578,
 'spentbut': 40579,
 'dieshe': 35584,
 'eugenics': 17041,
 'asianpacific': 40580,
 'strictest': 36358,
 'bringing': 2113,
 'seamier': 40582,
 'artwhich': 40583,
 'weell': 40584,
 'cloudthat': 40585,
 'wooded': 22559,
 'moviegrab': 40586,
 'liaisons': 2402

In [11]:
import pickle

In [15]:
with open('embeddings.pkl','wb') as f:
    pickle.dump({'embeddings' : embeddings.eval(session=session), 'dictionary' : dictionary},f)